<a href="https://colab.research.google.com/github/yadavrishikesh/BayesNF/blob/main/BayesNF_on_EcoCounter_bike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q bayesnf

In [3]:
!pip -q install cartopy
!pip -q install contextily
!pip -q install geopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 73.7 MB/s eta 0:00:00


In [4]:
import warnings
warnings.simplefilter('ignore')

import contextily as ctx
import geopandas as gpd
import jax
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from cartopy import crs as ccrs
from shapely.geometry import Point
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [7]:
#!wget -q https://cs.cmu.edu/~fsaad/assets/bayesnf/air_quality.5.train.csv
!wget -q https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/train_data_Eco_pred-type_spatIntpl.csv
df_train = pd.read_csv('train_data_Eco_pred-type_spatIntpl.csv', index_col=0, parse_dates=['datetime'])

In [12]:
df_train.head(10)

,datetime,elev,walkscore,num_ppo,temp,precp,visbl,wsp,lon,lat,response
8,2022-04-22,-0.579066,-0.533333,-0.021161,-0.691037,-0.070925,-0.126790,0.406899,-73.544426,45.530216,494
9,2022-04-23,-0.579066,-0.533333,-0.021161,-0.971799,-0.070925,0.487823,-0.360644,-73.544426,45.530216,677
10,2022-04-24,-0.579066,-0.533333,-0.021161,-0.796038,-0.070925,0.276179,0.508880,-73.544426,45.530216,474
11,2022-04-25,-0.579066,-0.533333,-0.021161,-0.218916,-0.070925,-0.331661,0.498145,-73.544426,45.530216,604
12,2022-04-26,-0.579066,-0.533333,-0.021161,-0.355112,-0.020347,-0.597486,0.261978,-73.544426,45.530216,620
13,2022-04-27,-0.579066,-0.533333,-0.021161,-1.025440,0.186361,-1.431362,0.420317,-73.544426,45.530216,198
14,2022-04-28,-0.579066,-0.533333,-0.021161,-0.989299,-0.070925,0.487823,0.490094,-73.544426,45.530216,743
15,2022-04-29,-0.579066,-0.533333,-0.021161,-0.778537,-0.070925,0.692694,0.310285,-73.544426,45.530216,720
16,2022-04-30,-0.579066,-0.533333,-0.021161,-0.603917,-0.066527,0.590258,-0.215723,-73.544426,45.530216,1056
17,2022-05-01,-0.579066,-0.533333,-0.021161,-0.323917,-0.070925,0.795129,-0.282816,-73.544426,45.530216,1387


In [ ]:
from bayesnf.spatiotemporal import BayesianNeuralFieldMAP

model = BayesianNeuralFieldMAP(
  width=512,
  depth=2,
  freq='H',
  seasonality_periods=['D', 'W'], # Daily and weekly seasonality, same as [24, 24*7]
  num_seasonal_harmonics=[4, 4], # Four harmonics for each seasonal factor.
  feature_cols=['datetime', 'latitude', 'longitude'], # time, spatial 1, ..., spatial n
  target_col='pm10',
  observation_model='NORMAL',
  timetype='index',
  standardize=['latitude', 'longitude'],
  )

In [ ]:
# Train MAP ensemble
model = model.fit(
    df_train,
    seed=jax.random.PRNGKey(0),
    ensemble_size=2,
    num_epochs=100,
    )